# Single Frame Model

This script trains a single frame model based on an average image taken across the the forcasted month.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, LayerNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
import time
import numpy as np

## 0. Define Region

First, define a region to train a model for.

In [2]:
REGIONS = ["ES","NS","WS","EM","EE","LD","NEE","NWE","SEE","SWE","WAL","WM","YH"]

## 1. Load Data

To begin, data for a single frame mnodel must be loaded along with the expected regional rainfall values.

### 1.0 New Training Data

This section loads the new training data set.

In [3]:
training_datafile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/single_all.npy"
training_rainfallfile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/expected_all.npy"

In [4]:
training_videos = np.load(training_datafile)
training_rainfall = np.load(training_rainfallfile)[:, 2:]

In [5]:
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)

In [6]:
training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))

for n in range(0, len(REGIONS)):
    training_rainfall[:, n] = (training_rainfall[:, n] - np.min(training_rainfall[:, n])) / (np.max(training_rainfall[:, n]) - np.min(training_rainfall[:, n]))

In [7]:
print(np.min(training_rainfall, axis=0))
print(np.max(training_rainfall, axis=0))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [8]:
def apply_augmentation(raw_rain, raw_vids):
    vids = np.copy(raw_vids)
    vids = vids + np.random.normal(0, 0.05, vids.shape)
    return np.concatenate((raw_vids, vids), axis=0), np.concatenate((raw_rain, raw_rain), axis=0)

APPLY_AUGMENTATION = True

if APPLY_AUGMENTATION:
    training_videos, training_rainfall = apply_augmentation(training_rainfall, training_videos)

In [9]:
training_videos.shape

(548, 61, 121, 2)

## 2. Model Definition

Next, a CNN model architecture is defined.

In [10]:
def model_generator(input_shape=(2, 61, 121), learning_rate=0.1):
    """ This method generates a model definition. """
    model = Sequential()
    
    # First layer
    model.add(Conv2D(8, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Second layer
    model.add(Conv2D(8, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Third layer
    model.add(Conv2D(8, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Final Layer
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('relu'))
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [11]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="D:/PHD_DATA/Video_18-01-2021/models/"):
    """ Trains the given model with the given dataset. """
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    history = model.fit(
        xdata,
        ydata,
        batch_size=8,
        validation_split=0.3,
        callbacks=[tensorboard, es],
        epochs=100
    )
    save_model(model, models_folder + run_name + ".mdl")
    return history

In [15]:
# Flexible parameters
learning_rate = 0.0001

In [ ]:
final_errors = {}
# Run model for each region
for ridx, r in enumerate(REGIONS):
    final_errors[r] = {}
    # Run each model multiple times
    for i in range(0, 3):
        run_name = "{}-{}".format(r, int(time.time()))
        tb = TensorBoard(log_dir="D:/PHD_DATA/Video_11-02-2021/SingleFrame/logs/{}".format(run_name))
        model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])
        history = train_model(run_name, tb, model, training_videos, training_rainfall[:, ridx],
                              models_folder="D:/PHD_DATA/Video_11-02-2021/SingleFrame/models/")
        final_errors[r][run_name] = history.history["val_loss"][-1]

Epoch 1/100
48/48 [==============================] - 7s 108ms/step - loss: 0.0822 - val_loss: 0.0314
Epoch 2/100
48/48 [==============================] - 2s 38ms/step - loss: 0.0284 - val_loss: 0.0307
Epoch 3/100
48/48 [==============================] - 2s 35ms/step - loss: 0.0300 - val_loss: 0.0306
Epoch 4/100
48/48 [==============================] - 2s 36ms/step - loss: 0.0310 - val_loss: 0.0303
Epoch 5/100
48/48 [==============================] - 2s 37ms/step - loss: 0.0279 - val_loss: 0.0310
Epoch 6/100
48/48 [==============================] - 2s 36ms/step - loss: 0.0250 - val_loss: 0.0310
Epoch 7/100
48/48 [==============================] - 2s 37ms/step - loss: 0.0281 - val_loss: 0.0299
Epoch 8/100
48/48 [==============================] - 2s 37ms/step - loss: 0.0264 - val_loss: 0.0297
Epoch 9/100
48/48 [==============================] - 2s 37ms/step - loss: 0.0286 - val_loss: 0.0304
Epoch 10/100
48/48 [==============================] - 2s 37ms/step - loss: 0.0249 - val_loss: 0.029